In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# from peft import get_peft_model, LoraConfig

# lora_rank = 64
# lora_alpha = 8

# lora_config = LoraConfig(
#     r=lora_rank,
#     lora_alpha=lora_alpha,
#     target_modules=["q_proj", "v_proj"],
# )

# model = get_peft_model(model, lora_config)

In [4]:
from unlearn_order.dataset import load_dataset

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 1
n_val = 1

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]
combined_files = train_files + val_files

train_dataset = load_dataset(data_dir, train_files)
val_dataset = load_dataset(data_dir, val_files)
combined_dataset = load_dataset(data_dir, combined_files)

In [5]:
from unlearn_order.pipeline import run_pipeline

batch_size = 4
tolerance = 0.01
lr = 3e-6
max_epochs = 100
eval_every = 5

run_pipeline(
    model,
    tokenizer,
    [
        ("f", "train_train", train_dataset),
        ("f", "val_train", val_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
        ("u", "unlearn", combined_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
        ("f", "retrain_train", train_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
    ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr,
    max_epochs=max_epochs,
)

  5%|▌         | 5/100 [00:59<20:05, 12.69s/it]

Epoch 5 loss: 0.06718318610434319 acc: 0.3630573248407643


 10%|█         | 10/100 [01:58<18:57, 12.64s/it]

Epoch 10 loss: 0.03730370041137193 acc: 0.7515923566878981


 15%|█▌        | 15/100 [02:57<17:55, 12.65s/it]

...

  5%|▌         | 5/100 [00:58<19:57, 12.61s/it]

Epoch 5 loss: 0.001023092105277298 acc: 0.9554140127388535


  9%|▉         | 9/100 [01:57<19:52, 13.11s/it]

Epoch 10 loss: 0.0008272518723298296 acc: 0.9936305732484076
task f.retrain_train done with accuracy 0.9936305732484076


task e.eval_train done with accuracy 0.9936305732484076
task e.eval_val done with accuracy 0.7707006369426752
